# 3. Deployment – Heart Disease Prediction Model (SVC)

This notebook demonstrates how to deploy the best-performing model  
(Support Vector Classifier tuned with hyperparameters).  

We will:
1. Upload dataset.
2. Retrain SVC with best parameters.
3. Save & reload the trained model.
4. Create a function for predictions.
---


In [1]:
# ==============================
# 1. Upload Dataset
# ==============================
from google.colab import files
import pandas as pd

uploaded = files.upload()  # Upload dataset (03_selected_features_dataset.csv)
df = pd.read_csv(next(iter(uploaded)))
df.head()


Saving 03_selected_features_dataset.csv to 03_selected_features_dataset.csv


,age,trestbps,chol,thalach,oldpeak,sex_0,sex_1,cp_0,cp_1,cp_2,...,restecg_2,exang_0,exang_1,slope_0,slope_1,ca_0,ca_2,thal_1,thal_3,target
0,0.948726,0.821446,-0.265040,0.015306,0.796300,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
1,1.392002,1.723905,0.851214,-1.835388,0.011015,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1
2,1.392002,-0.682652,-0.349285,-0.910041,1.090782,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1
3,-1.932564,-0.081013,0.093004,1.645679,1.974227,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
4,-1.489288,-0.081013,-0.875820,0.984717,-0.087146,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0


In [2]:
# ==============================
# 2. Split Data & Train Final SVC Model
# ==============================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import joblib

# Features & labels
X = df.drop("target", axis=1)
y = df["target"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Best SVC params from tuning (update with your actual best params)
best_svc = SVC(kernel="rbf", C=10, gamma=0.01, probability=True, random_state=42)
best_svc.fit(X_train, y_train)

# Evaluate
from sklearn.metrics import accuracy_score, roc_auc_score

y_pred = best_svc.predict(X_test)
y_proba = best_svc.predict_proba(X_test)[:,1]

print("Final SVC Test Accuracy:", accuracy_score(y_test, y_pred))
print("Final SVC Test AUC:", roc_auc_score(y_test, y_proba))


Final SVC Test Accuracy: 0.8688524590163934
Final SVC Test AUC: 0.9577922077922079


In [3]:
# ==============================
# 3. Save Model & Scaler
# ==============================
joblib.dump(best_svc, "svc_model.pkl")
joblib.dump(scaler, "scaler.pkl")
print("Model & Scaler Saved!")


Model & Scaler Saved!


In [4]:
# ==============================
# 4. Reload Model & Prediction Function
# ==============================
svc_loaded = joblib.load("svc_model.pkl")
scaler_loaded = joblib.load("scaler.pkl")

def predict_heart_disease(features_list):
    """
    features_list: list of patient values in same order as training features
    Example: [63, 1, 3, 145, 233, 1, 0, 150, 0, 2.3, 0, 0, 1]
    """
    data_scaled = scaler_loaded.transform([features_list])
    prediction = svc_loaded.predict(data_scaled)[0]
    proba = svc_loaded.predict_proba(data_scaled)[0][1]
    return prediction, proba

# Test with one sample from dataset
sample = X.iloc[0].tolist()
print("Prediction:", predict_heart_disease(sample))


Prediction: (np.int64(0), np.float64(0.174016670028161))


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
